![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [22]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', axis=1).plot()

In [40]:
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

import numpy as np
from matplotlib import pyplot as plt

qb = QuantBook()

asset = "SPY"

qb.AddEquity(asset, Resolution.Minute)

history = qb.History(qb.Securities.Keys, datetime(2019, 1, 1), datetime(2021, 12, 31), Resolution.Daily)

history

input_ = history[['high', 'low', 'open']].iloc[:-1].unstack(level=0)

scaler = MinMaxScaler(feature_range = (0, 1))
df = pd.DataFrame(scaler.fit_transform(input_), index=input_.index)

input_ = df


In [41]:
close_price = history['close'].unstack(level=0)
scaler = MinMaxScaler(feature_range = (0, 1))
df = pd.DataFrame(scaler.fit_transform(close_price), index=close_price.index)

df 


In [42]:
output = df.shift(-1).iloc[:-1]
output

In [43]:


splitter = int(input_.shape[0] * 0.8)
X_train = input_.iloc[:splitter]
X_test = input_.iloc[splitter:]
y_train = output.iloc[:splitter]
y_test = output.iloc[splitter:]

features_set = []
labels = []
for i in range(60, X_train.shape[0]):
    features_set.append(X_train.iloc[i-60:i].values.reshape(-1, 1))
    labels.append(y_train.iloc[i])
    
features_set, labels = np.array(features_set), np.array(labels)
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))


In [44]:
model = Sequential()

# Add our first LSTM layer - 50 nodes.
model.add(LSTM(units = 50, return_sequences=True, input_shape=(features_set.shape[1], 1)))
# Add Dropout layer to avoid overfitting
model.add(Dropout(0.2))
# Add additional layers
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units = 5))
model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mae', 'acc'])

callback = EarlyStopping(monitor='loss', patience=3, verbose=1, restore_best_weights=True)

model.summary()

In [45]:
model.fit(features_set, labels, epochs = 20, batch_size = 100, callbacks=[callback])

In [46]:
test_features = []
for i in range(60, X_test.shape[0]):
    test_features.append(X_test.iloc[i-60:i].values.reshape(-1, 1))
    
test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))

predictions = model.predict(test_features)

print(predictions)

predictions = scaler.inverse_transform(predictions)
actual = scaler.inverse_transform(y_test.values)

In [47]:
print(predictions)

In [48]:
plt.figure(figsize=(15, 10))
plt.plot(actual[60:], color='blue', label='Actual')
plt.plot(predictions , color='red', label='Prediction')
plt.title('Price vs Predicted Price ')
plt.legend()
plt.show()